In [1]:
# import torch
# from clip_validation_functions import CLIPS_VALIDATIONS  # Import your validation functions
# from base_validation_function import construct_tensor_validator  # Import the validator constructor

# # Define the column names in the order expected by the tensor
# COLUMN_NAMES = [
#     "Saddle height",
#     "Seat tube length",
#     "Seatpost LENGTH",
#     "BSD rear",
#     "ERD rear",
#     "Head tube length textfield",
#     "Head tube lower extension2",
#     "Head tube upper extension2",
#     "CS textfield"
# ]

# # Create the tensor-based validator
# tensor_validator = construct_tensor_validator(CLIPS_VALIDATIONS, COLUMN_NAMES)

# # Create a sample input tensor with requires_grad=True for gradient tracking
# sample_designs = torch.tensor(
#     [[500.0, 450.0, 100.0, 600.0, 580.0, 150.0, 20.0, 30.0, -10.0],  # Design 1
#      [520.0, 460.0, 105.0, 610.0, 590.0, 160.0, 25.0, 35.0, 5.0],    # Design 2
#      [480.0, 440.0, 95.0, 590.0, 570.0, 140.0, 15.0, 25.0, -5.0]],   # Design 3
#     dtype=torch.float32, requires_grad=True  # Enable gradient tracking
# )

# # Run validation
# validation_results = tensor_validator(sample_designs)

# # Compute a simple sum loss to check gradients
# # loss = validation_results.sum()
# loss = validation_results[validation_results > 0].sum()  # Sum only positive results
# loss.backward()

# # Display validation results
# print("Validation Results:")
# print(validation_results)

# # Display gradients
# print("\nGradients with respect to input tensor:")
# print(sample_designs.grad)


In [ ]:
import torch
from aero_validation_functions import AERO_VALIDATIONS  # Import your validation functions
from base_validation_function import construct_tensor_validator  # Import the validator constructor


# Define column names in the correct order
COLUMN_NAMES = [
    'hand_x', 'hand_y', 'hip_x', 'hip_y', 'crank_length', 
    'upper_leg', 'lower_leg', 'arm_length', 'torso_length', 
    'neck_and_head_length', 'torso_width'
]

# Create the tensor validator
tensor_validator = construct_tensor_validator(AERO_VALIDATIONS, COLUMN_NAMES)

def gradient_check(validator, num_samples=50):
    # Create a random input tensor with requires_grad=True
    sample_designs = torch.randn((num_samples, len(COLUMN_NAMES)), dtype=torch.float32, device='cuda', requires_grad=True)

    print(sample_designs.device)

    # Run the validation function
    validation_results = validator(sample_designs)

    print(validation_results.device)

    # Compute a simple loss function (sum of all positive validation outputs)
    loss = validation_results[validation_results > 0].sum()

    # Backpropagate
    loss.backward()

    # Display gradients
    print("\nGradients with respect to input tensor:")
    print(sample_designs.grad)
    print(sample_designs.grad.device)
    

# Run the gradient check
gradient_check(tensor_validator)


cuda:0
cuda:0

Gradients with respect to input tensor:
tensor([[ 0.0000e+00,  0.0000e+00,  9.9918e-01,  4.0559e-02,  3.0000e+00,
          0.0000e+00, -2.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-9.9998e-01,  6.3594e-03,  5.6879e+01,  1.9390e+02,  2.0436e+02,
         -4.6812e+00, -1.4820e+02, -1.0000e+00, -1.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-5.2893e-01, -8.4867e-01, -5.4654e+02, -3.5478e+02, -6.4962e+02,
          6.4446e+02,  6.2348e+02, -1.0000e+00, -1.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 6.3995e-01,  7.6842e-01,  6.3995e-01, -7.6842e-01,  0.0000e+00,
          0.0000e+00,  0.0000e+00, -1.0000e+00, -1.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-2.7705e+01, -2.7131e+01, -6.3480e+01,  3.7611e+01,  0.0000e+00,
          0.0000e+00,  0.0000e+00, -1.0000e+00, -1.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-9.4695e-01,  3.2137e-01,  1.0735e+02,  3.5007e+01,  1.1391e+02,
          1.0018

In [17]:
import torch
from aero_validation_functions import AERO_VALIDATIONS  # Import validation functions
from base_validation_function import construct_tensor_validator  # Import validator constructor

# Define column names in the correct order
COLUMN_NAMES = [
    'hand_x', 'hand_y', 'hip_x', 'hip_y', 'crank_length', 
    'upper_leg', 'lower_leg', 'arm_length', 'torso_length', 
    'neck_and_head_length', 'torso_width'
]

# Create the tensor validator
tensor_validator = construct_tensor_validator(AERO_VALIDATIONS, COLUMN_NAMES)

# Optimization parameters
num_samples = 5
learning_rate = 0.01
num_steps = 1000

def optimize_designs(validator, num_samples, learning_rate, num_steps):
    # Initialize random designs directly on CUDA with requires_grad=True
    designs = torch.randn((num_samples, len(COLUMN_NAMES)), dtype=torch.float32, device='cuda', requires_grad=True)

    # Use Adam optimizer for smooth updates
    optimizer = torch.optim.Adam([designs], lr=learning_rate)

    for step in range(num_steps):
        # Compute validation function values (constraints)
        validation_results = validator(designs)

        # Compute loss as sum of constraint values
        loss = validation_results[validation_results > 0].sum()

        # Perform gradient descent
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print progress every 10 steps
        if step % 10 == 0:
            avg_loss = loss.item() / num_samples
            print(f"Step {step}: Avg Constraint Value = {avg_loss:.4f}")

    return designs

# Run optimization
optimized_designs = optimize_designs(tensor_validator, num_samples, learning_rate, num_steps)

# Display final designs
print("\nFinal Optimized Designs:")
print(optimized_designs)


Step 0: Avg Constraint Value = 68.1533
Step 10: Avg Constraint Value = 77.3007
Step 20: Avg Constraint Value = 67.0287
Step 30: Avg Constraint Value = 66.6069
Step 40: Avg Constraint Value = 97.6990
Step 50: Avg Constraint Value = 26.9785
Step 60: Avg Constraint Value = 16.7099
Step 70: Avg Constraint Value = 5.0237
Step 80: Avg Constraint Value = 60.6605
Step 90: Avg Constraint Value = 53.0950
Step 100: Avg Constraint Value = 35.8863
Step 110: Avg Constraint Value = 23.9326
Step 120: Avg Constraint Value = 10.3453
Step 130: Avg Constraint Value = 0.7972
Step 140: Avg Constraint Value = 0.7282
Step 150: Avg Constraint Value = 0.6564
Step 160: Avg Constraint Value = 0.5834
Step 170: Avg Constraint Value = 0.5670
Step 180: Avg Constraint Value = 0.5577
Step 190: Avg Constraint Value = 0.5481
Step 200: Avg Constraint Value = 0.5381
Step 210: Avg Constraint Value = 0.5280
Step 220: Avg Constraint Value = 0.5175
Step 230: Avg Constraint Value = 0.5068
Step 240: Avg Constraint Value = 0.4958

In [20]:
import pandas as pd
df = pd.DataFrame(optimized_designs.cpu().detach().numpy(), columns=COLUMN_NAMES)
display(df)

,hand_x,hand_y,hip_x,hip_y,crank_length,upper_leg,lower_leg,arm_length,torso_length,neck_and_head_length,torso_width
0,-1.146184,-0.900655,1.086311,-0.904523,0.142117,1.749617,0.499385,-0.174679,0.400686,-0.067766,-0.059156
1,0.726014,0.459814,0.775969,-0.013494,-0.047211,1.195126,0.396204,0.715557,1.064327,-0.634273,-1.029181
2,0.296420,0.646043,-0.115630,0.515780,-0.880840,0.288929,-0.955919,-1.547224,2.078327,0.639549,0.655578
3,0.837924,0.605561,-0.617532,0.514910,-0.300752,0.436167,1.527580,0.865921,0.185688,0.240900,-0.096351
4,0.925874,0.240767,-0.476769,0.560528,-1.351400,0.476808,-0.108342,-0.954731,0.933162,-0.027476,-0.126161


In [25]:
tensor_validator(optimized_designs)

tensor([[-6.9330e-01, -1.6601e-01, -2.1240e-02, -3.6633e+02, -5.7777e+00],
        [-8.6245e-01, -2.0509e-01, -2.4376e-02, -1.8630e+02, -3.3844e+00],
        [ 3.1473e-01, -3.0827e-01, -1.6457e-01, -1.0314e+02, -2.0000e+01],
        [-1.4605e+00, -8.1330e-01, -1.3378e-02, -5.6909e+01, -7.9463e+00],
        [-9.8400e-01,  5.7288e-01, -1.5021e+00, -7.9343e+01, -2.0000e+01]],
       device='cuda:0', grad_fn=<CopySlices>)